In [15]:
import yaml
import pandas as pd
from graphdatascience import GraphDataScience

In [16]:
with open('config.yaml', 'r') as file:
    PARAM = yaml.safe_load(file)


url = "bolt://localhost:7687"
gds = GraphDataScience(url, auth=(PARAM["neo4j_username"], PARAM["neo4j_password"]))

In [17]:
G, result = gds.graph.project(
    "contextGraph",                  #  Graph name
    {
    "Known": { "properties":['protection_status_one_hot', 'site_area_in_hectares', 'site_length_km', 'valuation_methods_one_hot', 'value_year', 'beneficiary_unit_one_hot', 'int__per_hectare_per_year'] },
    "Unknown": {"properties":['protection_status_one_hot', 'site_area_in_hectares', 'site_length_km', 'valuation_methods_one_hot', 'value_year', 'beneficiary_unit_one_hot']},
    "Country": {},
    "Seea": {},
    "Ecosystem": {},
    "Ecozone": {},
    "Biome": {},
    "Cices_class": {},
    "Cices_division": {},
    "Cices_group": {},
    "Cices_section": {},
    "Teeb_service": {},
    "Teeb_subservice": {},
    "Teeb_category": {},
    "Subregion": {},
    "Region": {}
  },                 #  Node projection
  {
    "IS_LOCATE_IN": {"orientation": 'UNDIRECTED'},
    "IS_AN_SEEA_OF": {"orientation": 'UNDIRECTED'},
    "IS_AN_ECOSYSTEM_OF": {"orientation": 'UNDIRECTED'},
    "ECOSYSTEM_BELONGS_TO": {"orientation": 'UNDIRECTED'},
    "IS_AN_ECOZONE_OF": {"orientation": 'UNDIRECTED'},
    "ECOZONE_BELONGS_TO": {"orientation": 'UNDIRECTED'},
    "IS_A_BIOME_OF": {"orientation": 'UNDIRECTED'},
    "BELONGS_TO_CLASS_OF_CICES": {"orientation": 'UNDIRECTED'},
    "IS_A_GROUP_OF_CICES": {"orientation": 'UNDIRECTED'},
    "IS_A_DIVISION_OF_CICES": {"orientation": 'UNDIRECTED'},
    "IS_A_CLASS_OF_CICES": {"orientation": 'UNDIRECTED'},
    "PROVIDES_SERVICE": {"orientation": 'UNDIRECTED'},
    "PROVIDES_SUBSERVICE": {"orientation": 'UNDIRECTED'},
    "COUNTRY_IS_LOCATE_IN": {"orientation": 'UNDIRECTED'},
    "SUBREGION_IS_LOCATE_IN": {"orientation": 'UNDIRECTED'},
    "IS_A_SUBSERVICE_OF": {"orientation": 'UNDIRECTED'},
    "IS_A_SERVICE_OF":  {"orientation": 'UNDIRECTED'}     
    },              #  Relationship projection
    readConcurrency=4           #  Configuration parameters
)

In [18]:
pipe_with_context = gds.nr_pipe("pipe-with-context")

In [19]:
pipe_with_context.addNodeProperty(
    "hashgnn",
    mutateProperty="embedding",
    iterations=10,
    embeddingDensity=32,
    generateFeatures={"dimension": 8, "densityLevel": 3},
    contextNodeLabels=['Country', 'Seea', 'Ecosystem', 'Ecozone', 'Ecosystem', 'Biome','Cices_class', 'Teeb_service', 'Teeb_subservice', 'Subregion', 'Region'],
)

name                                                 pipe-with-context
nodePropertySteps    [{'name': 'gds.hashgnn.mutate', 'config': {'em...
featureProperties                                                   []
splitConfig                {'testFraction': 0.3, 'validationFolds': 3}
autoTuningConfig                                     {'maxTrials': 10}
parameterSpace            {'LinearRegression': [], 'RandomForest': []}
Name: 0, dtype: object

In [20]:
pipe_with_context.selectFeatures(["embedding"])

name                                                 pipe-with-context
nodePropertySteps    [{'name': 'gds.hashgnn.mutate', 'config': {'em...
featureProperties                                          [embedding]
splitConfig                {'testFraction': 0.3, 'validationFolds': 3}
autoTuningConfig                                     {'maxTrials': 10}
parameterSpace            {'LinearRegression': [], 'RandomForest': []}
Name: 0, dtype: object

In [21]:
pipe_with_context.addRandomForest(numberOfDecisionTrees=10)

name                                                 pipe-with-context
nodePropertySteps    [{'name': 'gds.hashgnn.mutate', 'config': {'em...
featureProperties                                          [embedding]
splitConfig                {'testFraction': 0.3, 'validationFolds': 3}
autoTuningConfig                                     {'maxTrials': 10}
parameterSpace       {'LinearRegression': [], 'RandomForest': [{'ma...
Name: 0, dtype: object

In [22]:
pipe_with_context.configureAutoTuning(maxTrials=5)

name                                                 pipe-with-context
nodePropertySteps    [{'name': 'gds.hashgnn.mutate', 'config': {'em...
featureProperties                                          [embedding]
splitConfig                {'testFraction': 0.3, 'validationFolds': 3}
autoTuningConfig                                      {'maxTrials': 5}
parameterSpace       {'LinearRegression': [], 'RandomForest': [{'ma...
Name: 0, dtype: object

In [23]:
model, stats = pipe_with_context.train(
    G,
    targetNodeLabels=["Known"],
    modelName="nr-pipeline-model-contextual",
    targetProperty="int__per_hectare_per_year",
    metrics=["ROOT_MEAN_SQUARED_ERROR"],
    concurrency=4
)

Node Regression Train Pipeline:   0%|          | 0/100 [00:00<?, ?%/s]

In [24]:
print (stats["modelInfo"]["metrics"]["ROOT_MEAN_SQUARED_ERROR"])

{'test': 53366.017671835354, 'validation': {'min': 38913.5902903431, 'max': 63629.37373211389, 'avg': 48677.71448588896}, 'outerTrain': 50567.92370373522, 'train': {'min': 44512.68830434047, 'max': 53648.44344772046, 'avg': 50064.89449302853}}


In [25]:
predicted = model.predict_stream(
    G, modelName="nr-pipeline-model-contextual", targetNodeLabels=["Unknown"]
)
     


Node Classification Predict Pipeline:   0%|          | 0/100 [00:00<?, ?%/s]

In [26]:
predicted["name"] = [x["name"] for x in gds.util.asNodes(predicted["nodeId"].to_list())]

In [27]:
predicted.rename(columns={"predictedValue": "HashGNN_rf"})

,nodeId,HashGNN_rf,name
0,23,2675.5137,12270007
1,31,2675.5137,12250001
2,35,2675.5137,12240002
3,41,2675.5137,12230006
4,46,2675.5137,12210004
...,...,...,...
3471,10291,2675.5137,12410007
3472,10293,2675.5137,12400002
3473,10308,2675.5137,12360001
3474,10315,2675.5137,12350007


In [28]:
predicted.to_csv("predicted_HashGNN.tsv", sep="\t", index=False)